<a href="https://colab.research.google.com/github/Duvhier/obesidadanalisisdedatos/blob/main/Obesidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import itertools
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [18]:
DatasetName = 'https://archive.ics.uci.edu/static/public/544/estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition.zip'
df = pd.read_csv(DatasetName, sep=',')

#df.head()

print(df)

df.describe()


      Gender        Age    Height      Weight family_history_with_overweight  \
0     Female  21.000000  1.620000   64.000000                            yes   
1     Female  21.000000  1.520000   56.000000                            yes   
2       Male  23.000000  1.800000   77.000000                            yes   
3       Male  27.000000  1.800000   87.000000                             no   
4       Male  22.000000  1.780000   89.800000                             no   
...      ...        ...       ...         ...                            ...   
2106  Female  20.976842  1.710730  131.408528                            yes   
2107  Female  21.982942  1.748584  133.742943                            yes   
2108  Female  22.524036  1.752206  133.689352                            yes   
2109  Female  24.361936  1.739450  133.346641                            yes   
2110  Female  23.664709  1.738836  133.472641                            yes   

     FAVC  FCVC  NCP       CAEC SMOKE  

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
count,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000,2111.000000
mean,24.312600,1.701677,86.586058,2.419043,2.685628,2.008011,1.010298,0.657866
std,6.345968,0.093305,26.191172,0.533927,0.778039,0.612953,0.850592,0.608927
min,14.000000,1.450000,39.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,19.947192,1.630000,65.473343,2.000000,2.658738,1.584812,0.124505,0.000000
50%,22.777890,1.700499,83.000000,2.385502,3.000000,2.000000,1.000000,0.625350
75%,26.000000,1.768464,107.430682,3.000000,3.000000,2.477420,1.666678,1.000000
max,61.000000,1.980000,173.000000,3.000000,4.000000,3.000000,3.000000,2.000000


In [19]:
import numpy as np
import pandas as pd

# Diccionario para mapear los valores de la columna 'NObeyesdad' a números
class_map = {
    "Normal_Weight": 1,
    "Overweight_Level_I": 2,
    "Overweight_Level_II": 3,
    "Obesity_Type_I": 4,
    "Obesity_Type_II": 5,
    "Obesity_Type_III": 6,
    "Insufficient_Weight": 7
}

# Asignamos los números a la columna 'NObeyesdad'
df['NObeyesdad_Num'] = df['NObeyesdad'].map(class_map)

# Verificamos si la transformación fue correcta
print(df[['NObeyesdad', 'NObeyesdad_Num']])

# Si necesitas la columna NObeyesdad_Num como Y (las etiquetas de clase):
Y = df['NObeyesdad_Num'].values

# Mostramos las clases numéricas en Y
print("Clases en Y:", Y)




               NObeyesdad  NObeyesdad_Num
0           Normal_Weight               1
1           Normal_Weight               1
2           Normal_Weight               1
3      Overweight_Level_I               2
4     Overweight_Level_II               3
...                   ...             ...
2106     Obesity_Type_III               6
2107     Obesity_Type_III               6
2108     Obesity_Type_III               6
2109     Obesity_Type_III               6
2110     Obesity_Type_III               6

[2111 rows x 2 columns]
Clases en Y: [1 1 1 ... 6 6 6]


In [20]:
# Número de muestras totales (filas y columnas)
NumSamplesY = df.shape[0]
print('Número de muestras totales: ', NumSamplesY)
NumSamplesX = df.shape[1]
print('Número de características por muestra: ', NumSamplesX)

# Filtrando muestras por clase en la columna 'NObeyesdad'
dataClass1 = df[df['NObeyesdad_Num'] == 1]
dataClass2 = df[df['NObeyesdad_Num'] == 2]
dataClass3 = df[df['NObeyesdad_Num'] == 3]
dataClass4 = df[df['NObeyesdad_Num'] == 4]
dataClass5 = df[df['NObeyesdad_Num'] == 5]
dataClass6 = df[df['NObeyesdad_Num'] == 6]
dataClass7 = df[df['NObeyesdad_Num'] == 7]

# Número de muestras por clase
NumSamplesClass1 = dataClass1.shape[0]
NumSamplesClass2 = dataClass2.shape[0]
NumSamplesClass3 = dataClass3.shape[0]
NumSamplesClass4 = dataClass4.shape[0]
NumSamplesClass5 = dataClass5.shape[0]
NumSamplesClass6 = dataClass6.shape[0]
NumSamplesClass7 = dataClass7.shape[0]

# Imprimiendo los resultados
print('Número de muestras por clase 1 (Peso Normal): ', NumSamplesClass1,
      ' (', round(100 * NumSamplesClass1 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 2 (Sobrepeso Nivel 1): ', NumSamplesClass2,
      ' (', round(100 * NumSamplesClass2 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 3 (Sobrepeso Nivel 2): ', NumSamplesClass3,
      ' (', round(100 * NumSamplesClass3 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 4 (Obesidad Tipo 1): ', NumSamplesClass4,
      ' (', round(100 * NumSamplesClass4 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 5 (Obesidad Tipo 2): ', NumSamplesClass5,
      ' (', round(100 * NumSamplesClass5 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 6 (Obesidad Tipo 3): ', NumSamplesClass6,
      ' (', round(100 * NumSamplesClass6 / NumSamplesY, 2), '%)')
print('Número de muestras por clase 7 (Bajo Peso): ', NumSamplesClass7,
      ' (', round(100 * NumSamplesClass7 / NumSamplesY, 2), '%)')



Número de muestras totales:  2111
Número de características por muestra:  18
Número de muestras por clase 1 (Peso Normal):  287  ( 13.6 %)
Número de muestras por clase 2 (Sobrepeso Nivel 1):  290  ( 13.74 %)
Número de muestras por clase 3 (Sobrepeso Nivel 2):  290  ( 13.74 %)
Número de muestras por clase 4 (Obesidad Tipo 1):  351  ( 16.63 %)
Número de muestras por clase 5 (Obesidad Tipo 2):  297  ( 14.07 %)
Número de muestras por clase 6 (Obesidad Tipo 3):  324  ( 15.35 %)
Número de muestras por clase 7 (Bajo Peso):  272  ( 12.88 %)


In [31]:
import numpy as np

# Configurar NumPy para que muestre todo el contenido del array sin truncar
np.set_printoptions(threshold=np.inf)

# Concatenar los datos de cada clase, incluyendo la nueva columna 'NObeyesdad_Num'
dfAsNumpy = np.concatenate((dataClass1.values, dataClass2.values, dataClass3.values,
                            dataClass4.values, dataClass5.values, dataClass6.values,
                            dataClass7.values, ), axis=0)

# Selección de características (columnas 1, 2, 3)
X = dfAsNumpy[:, [1, 2, 3]]

# Selección de la columna que contiene las clases (suponiendo que es la última columna)
# Cambia el 17 por el índice real si la columna de clase está en otro lugar.
Y = dfAsNumpy[:, -1]

# Mostramos las dimensiones de X e Y para confirmar
print(X.shape)
print(Y.shape)

# Formateamos la salida
# Mostramos X e Y para verificar el contenido
# Columna 1 (Edad) sin decimales, Columna 2 (Altura) y Columna 3 (Peso) con 2 decimales
output = "X: ("
for row in X:
    output += f"Edad: {int(row[0])}, Altura: {row[1]:.2f}, Peso: {row[2]:.2f}\n"

output = output.strip() + ")"
print(output)
print("Y:", Y)



(2111, 3)
(2111,)
X: (Edad: 21, Altura: 1.62, Peso: 64.00
Edad: 21, Altura: 1.52, Peso: 56.00
Edad: 23, Altura: 1.80, Peso: 77.00
Edad: 29, Altura: 1.62, Peso: 53.00
Edad: 23, Altura: 1.50, Peso: 55.00
Edad: 22, Altura: 1.64, Peso: 53.00
Edad: 24, Altura: 1.78, Peso: 64.00
Edad: 22, Altura: 1.72, Peso: 68.00
Edad: 22, Altura: 1.65, Peso: 56.00
Edad: 23, Altura: 1.77, Peso: 60.00
Edad: 22, Altura: 1.70, Peso: 66.00
Edad: 22, Altura: 1.65, Peso: 60.00
Edad: 21, Altura: 1.85, Peso: 68.00
Edad: 20, Altura: 1.60, Peso: 50.00
Edad: 21, Altura: 1.70, Peso: 65.00
Edad: 23, Altura: 1.60, Peso: 52.00
Edad: 19, Altura: 1.75, Peso: 76.00
Edad: 23, Altura: 1.68, Peso: 70.00
Edad: 24, Altura: 1.77, Peso: 76.00
Edad: 22, Altura: 1.65, Peso: 62.00
Edad: 22, Altura: 1.56, Peso: 49.00
Edad: 21, Altura: 1.60, Peso: 48.00
Edad: 23, Altura: 1.65, Peso: 67.00
Edad: 23, Altura: 1.68, Peso: 60.00
Edad: 21, Altura: 1.66, Peso: 64.00
Edad: 21, Altura: 1.66, Peso: 62.00
Edad: 21, Altura: 1.81, Peso: 80.00
Edad: 

In [33]:
# Configurar NumPy para que muestre todo el contenido del array sin truncar
np.set_printoptions(threshold=np.inf)

# Extraer los valores de la columna 'NObeyesdad_Num'
Y = df['NObeyesdad_Num'].values

# Instanciar el LabelBinarizer
lb = preprocessing.LabelBinarizer()

# Ajustar el binarizador con las clases únicas de la columna NObeyesdad
lb.fit(Y)

# Mostrar el formato original de Y
print('Salida Y Antes: ', Y.shape)
print(Y)
print("")

# Transformar Y a su forma binarizada
Y2 = lb.transform(Y)

# Mostrar el nuevo formato de Y
print('Salida Y Después: ', Y2.shape)
print(Y2)


Salida Y Antes:  (2111,)
[1 1 1 2 3 1 1 1 1 1 4 3 1 4 1 1 3 4 3 2 3 4 1 4 1 1 1 1 1 1 2 3 1 3 1 3 1
 1 1 3 2 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 7 1 1 1 1 1 2 3 4 5 1 3 7 1 1
 3 7 7 3 4 1 1 3 4 7 3 1 1 2 1 3 5 1 2 1 1 1 1 7 7 1 1 1 1 1 4 1 1 2 4 4 1
 1 1 1 1 1 4 3 1 3 2 3 7 1 3 2 1 1 1 3 1 1 1 1 4 2 1 4 1 1 7 1 4 2 2 2 1 4
 1 4 1 1 2 3 4 1 1 1 2 3 1 3 1 1 3 5 1 3 2 3 3 1 1 1 2 1 1 1 1 2 1 1 7 3 2
 2 4 4 4 1 1 2 2 1 4 2 1 5 7 1 4 4 6 4 1 4 4 1 1 1 5 1 3 1 1 1 1 2 1 2 2 3
 1 1 4 5 1 1 3 5 2 1 1 3 1 1 1 1 1 1 1 2 1 3 1 7 1 1 1 2 3 3 3 1 1 2 3 4 3
 4 2 2 1 7 1 7 7 3 1 1 4 1 1 1 1 1 7 1 1 7 1 1 1 1 1 4 1 1 1 1 1 7 1 1 4 1
 1 2 1 2 1 1 7 1 2 5 1 1 1 1 7 1 1 1 1 1 1 1 1 2 1 1 7 1 2 1 1 2 1 4 7 1 1
 1 2 7 1 1 2 7 1 1 4 1 6 1 1 3 1 4 1 1 1 1 1 2 7 1 5 1 1 4 2 1 2 1 3 2 3 3
 1 2 1 1 1 3 1 1 1 1 3 1 1 4 1 1 1 4 4 2 7 7 1 3 1 7 1 1 5 1 2 1 3 6 1 3 1
 2 1 1 1 2 1 3 4 1 2 3 4 4 7 7 1 3 2 1 1 1 1 1 4 1 1 1 1 2 1 1 1 1 1 1 1 7
 1 1 1 2 5 1 1 1 1 1 1 1 1 1 1 2 1 4 3 1 1 1 1 1 7 7 1 4 3 1 1 2 2 1 1 1 1


In [34]:
#Algortimo para resolver optimización del entrenamiento
#solver = 'sgd'
#solver = 'adam'
solver = 'lbfgs'

#Número de Capas Ocultas y Neuronas por Capa
hidden_layer_sizes=(10)

#Función de activación
activation = 'logistic'
#activation = 'tanh'

#Máximo número de iteraciones para entrenamiento
#max_iter = 5
max_iter = 2011

#Construcción de la RNA con base en la arquitectura definida
clf = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, max_iter=max_iter, alpha=0.0001, activation=activation,
                     solver=solver, verbose=10,  random_state=21,tol=0.000000001)

#Entrenamiento y validación cruzada
#scores = cross_val_score(clf, X, Y, cv=1)

#Entrenamiento detallado
#Partición de entrenamiento y pruebas en 80%-20%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y2, test_size=0.20)

#Entrenamiento
clf.fit(X_train,Y_train)

#Predicción
Y_predicted = clf.predict(X_test)

#Comparación
Total = 43
Aciertos = 0
for i in range(0,43):
  print(Y_test[i], "vs", Y_predicted[i])
  if Y_test[i,0] == Y_predicted[i,0] and Y_test[i,1] == Y_predicted[i,1] and Y_test[i,2] == Y_predicted[i,2]:
    Aciertos+=1
print("Porcentaje: ", Aciertos/Total)

#Matriz de confusión
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test, Y_predicted)



[0 1 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 0 0 0 1] vs [0 0 0 0 0 0 0]
[0 0 0 0 1 0 0] vs [0 0 0 0 1 0 0]
[0 0 0 0 0 0 1] vs [0 0 0 0 0 0 0]
[0 0 0 0 0 0 1] vs [0 0 0 0 0 0 0]
[0 0 0 0 1 0 0] vs [0 0 0 0 1 0 0]
[0 1 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 1 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 1 0 0 0 0] vs [0 0 0 0 0 0 0]
[1 0 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[1 0 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 1 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 1 0 0 0] vs [0 0 0 0 1 0 0]
[0 0 0 0 0 1 0] vs [0 0 0 0 0 1 0]
[0 1 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[1 0 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[1 0 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 0 1 0 0] vs [0 0 0 0 1 0 0]
[0 1 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 0 1 0 0] vs [0 0 0 0 1 0 0]
[0 0 0 1 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 1 0 0 0] vs [0 0 0 0 0 0 0]
[0 1 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 1 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 1 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 0 0 0 1] vs [0 0 0 0 0 0 0]
[1 0 0 0 0 0 0] vs [0 0 0 0 0 0 0]
[0 0 0 0 1 0 0] vs [0 0 0 0 1 0 0]
[0 0 0 0 0 1 0] vs [

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:545: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


array([[[372,   0],
        [ 51,   0]],

       [[367,   0],
        [ 56,   0]],

       [[353,   0],
        [ 70,   0]],

       [[353,   0],
        [ 70,   0]],

       [[340,  25],
        [  6,  52]],

       [[364,   1],
        [  6,  52]],

       [[363,   0],
        [ 60,   0]]])

In [35]:
from sklearn import svm
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
clf2 = svm.SVC(kernel="linear")
clf2.fit(X_train,Y_train)
Y_predicted2 = clf2.predict(X_test)

#Comparación
Total = 43
Aciertos = 0
for i in range(0,43):
  print(Y_test[i], "vs", Y_predicted2[i])
  #if Y_test[i,0] == Y_predicted[i,0] and Y_test[i,1] == Y_predicted[i,1] and Y_test[i,2] == Y_predicted[i,2]:
  if Y_test[i] == Y_predicted2[i]:
    Aciertos+=1
print("Porcentaje: ", Aciertos/Total)

#Matriz de confusión
from sklearn.metrics import multilabel_confusion_matrix
multilabel_confusion_matrix(Y_test, Y_predicted2)


7 vs 2
1 vs 2
2 vs 2
1 vs 1
7 vs 1
3 vs 4
6 vs 6
6 vs 6
3 vs 7
6 vs 6
4 vs 4
6 vs 6
7 vs 2
6 vs 6
7 vs 7
1 vs 6
1 vs 6
3 vs 6
6 vs 6
5 vs 5
3 vs 3
3 vs 6
3 vs 3
5 vs 5
7 vs 1
7 vs 2
4 vs 4
6 vs 6
5 vs 5
2 vs 3
7 vs 7
7 vs 2
4 vs 1
6 vs 6
2 vs 2
2 vs 2
5 vs 5
5 vs 5
5 vs 5
4 vs 4
1 vs 6
3 vs 3
7 vs 7
Porcentaje:  0.627906976744186


array([[[332,  29],
        [ 26,  36]],

       [[322,  50],
        [ 31,  20]],

       [[349,  16],
        [ 38,  20]],

       [[337,  18],
        [ 19,  49]],

       [[349,  15],
        [  4,  55]],

       [[328,  36],
        [ 11,  48]],

       [[343,  14],
        [ 49,  17]]])